In [1]:
##IMPORT 
import numpy as np
from tqdm import tqdm, trange
import os
from glob import glob
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from PIL import Image


In [2]:
from einops import rearrange
# suppose we have a set of 32 images in "h w c" format (height-width-channel)
images = [np.random.randn(30, 40, 3) for _ in range(32)]

# stack along first (batch) axis, output is a single array
rearrange(images, 'b h w c -> b h w c').shape
(32, 30, 40, 3)

# concatenate images along height (vertical axis), 960 = 32 * 30
rearrange(images, 'b h w c -> (b h) w c').shape
(960, 40, 3)

# concatenated images along horizontal axis, 1280 = 32 * 40
rearrange(images, 'b h w c -> h (b w) c').shape
(30, 1280, 3)

# reordered axes to "b c h w" format for deep learning
rearrange(images, 'b h w c -> b c h w').shape
(32, 3, 30, 40)

# flattened each image into a vector, 3600 = 30 * 40 * 3
rearrange(images, 'b h w c -> b (c h w)').shape
(32, 3600)

# split each image into 4 smaller (top-left, top-right, bottom-left, bottom-right), 128 = 32 * 2 * 2
rearrange(images, 'b (h1 h) (w1 w) c -> (b h1 w1) h w c', h1=2, w1=2).shape
(128, 15, 20, 3)

# space-to-depth operation
rearrange(images, 'b (h h1) (w w1) c -> b h w (c h1 w1)', h1=2, w1=2).shape
(32, 15, 20, 12)

(32, 15, 20, 12)

In [23]:
qkv=torch.rand(1,65,6144)
head_num=4

query, key, value = tuple(rearrange(qkv, 'b t (d k h ) -> k b h t d ', k=3, h=head_num))
print(value.shape)

energy = torch.einsum("... i d , ... j d -> ... i j", query, key) 
print(energy.shape)

torch.Size([1, 4, 65, 512])
torch.Size([1, 4, 65, 65])


In [38]:
rand_tensor     =   torch.tensor([[[[3,1],
                                    [7,2],
                                    [4,4]],
                                   [[2,5],
                                    [4,3],
                                    [1,5]],
                                   [[6,4],
                                    [1,3],
                                    [7,4]],
                                   [[3,2],
                                    [3,1],
                                    [5,3]]
                                    ]])

energy = torch.einsum("... i d , ... j d -> ... i j", rand_tensor, rand_tensor) 
print(rand_tensor,rand_tensor.shape)
print(energy,energy.shape)
torch.matmul(rand_tensor, rand_tensor.T)

tensor([[[[3, 1],
          [7, 2],
          [4, 4]],

         [[2, 5],
          [4, 3],
          [1, 5]],

         [[6, 4],
          [1, 3],
          [7, 4]],

         [[3, 2],
          [3, 1],
          [5, 3]]]]) torch.Size([1, 4, 3, 2])
tensor([[[[10, 23, 16],
          [23, 53, 36],
          [16, 36, 32]],

         [[29, 23, 27],
          [23, 25, 19],
          [27, 19, 26]],

         [[52, 18, 58],
          [18, 10, 19],
          [58, 19, 65]],

         [[13, 11, 21],
          [11, 10, 18],
          [21, 18, 34]]]]) torch.Size([1, 4, 3, 3])


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 1

In [45]:
rand_tensor=torch.tensor([[3, 1],
                [7, 2],
                [4, 4]])
print(rand_tensor.shape)
torch.matmul(rand_tensor, rand_tensor.T)

torch.Size([3, 2])


tensor([[10, 23, 16],
        [23, 53, 36],
        [16, 36, 32]])